# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Load crime data set into pandas
city_weather_p2_df = pd.read_csv("/Users/theluggage/python-api-challenge/output_data/city_weather.csv")
city_weather_p2_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Pochutla,15.7432,-96.4661,88.83,59,99,7.54,MX,1635106966
1,1,Bluff,-46.6000,168.3333,52.86,92,92,5.99,NZ,1635106718
2,2,Saint Paul Harbor,57.7900,-152.4072,42.93,86,30,7.78,US,1635106967
3,3,Airai,-8.9266,125.4092,61.09,86,98,1.39,TL,1635106889
4,4,Avarua,-21.2078,-159.7750,73.45,68,75,11.50,CK,1635106797


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

# Create variables (locations)
locations = city_weather_p2_df[["Lat", "Lng"]]
weight = city_weather_p2_df["Humidity"]

locations
weight

0      59
1      92
2      86
3      86
4      68
       ..
576    83
577    44
578    53
579    84
580    65
Name: Humidity, Length: 581, dtype: int64

In [6]:

#Get max_intensity
humidity_max = city_weather_p2_df["Humidity"].max()
max_intensity = humidity_max

# #Add heatmap layer to the map.
# heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=humidity_max, point_radius=3)

# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=humidity_max,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)
fig

# # Customize the size of the figure
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }

# fig = gmaps.figure(layout=figure_layout)
# # Assign the marker layer to a variable
# markers = gmaps.marker_layer(locations)
# # Add the layer to the map
# fig.add_layer(markers)
# fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Create data frame that conforms to my (albeit very undefined) ideal weather conditions, which are:
    #A max temperature lower than 92 degrees but higher than 70.
    #Wind speed less than 10 mph.
    #Less than 80% cloudiness.
    #No requirements for humidity.

ideal_weather_df = city_weather_p2_df.loc[(city_weather_p2_df["Max Temp"] >= 70) & (city_weather_p2_df["Max Temp"] <= 92) & \
                                   (city_weather_p2_df["Wind Speed"] <= 10) & (city_weather_p2_df["Cloudiness"] <= 80)].dropna()

ideal_weather_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,Bandarbeyla,9.4942,50.8122,78.62,78,27,8.43,SO,1635106969
10,10,Oriximiná,-1.7656,-55.8661,88.61,56,73,4.32,BR,1635106970
27,27,Kapaa,22.0752,-159.3190,80.92,75,75,1.01,US,1635106887
31,31,Hilo,19.7297,-155.0900,79.29,88,45,4.00,US,1635106852
50,50,Port Blair,11.6667,92.7500,79.92,80,68,4.59,IN,1635106985
...,...,...,...,...,...,...,...,...,...,...
545,545,Esim,4.8699,-2.2405,78.06,86,75,7.20,GH,1635107188
558,558,La Palma,33.8464,-118.0467,76.12,65,40,3.00,US,1635107192
563,563,Aswān,24.0934,32.9070,76.30,27,0,9.22,EG,1635107056
564,564,Kingsport,36.5484,-82.5618,79.03,41,40,3.00,US,1635107195


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Prep dataframe.

# Create a hotel_df
hotel_df = ideal_weather_df

# Add a column for Hotel Name and drop any columns that do not correspond with the location.
hotel_df['Hotel Name'] = ""
hotel_df.drop(columns=['Unnamed: 0', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Date'])

hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,Bandarbeyla,9.4942,50.8122,78.62,78,27,8.43,SO,1635106969,
10,10,Oriximiná,-1.7656,-55.8661,88.61,56,73,4.32,BR,1635106970,
27,27,Kapaa,22.0752,-159.3190,80.92,75,75,1.01,US,1635106887,
31,31,Hilo,19.7297,-155.0900,79.29,88,45,4.00,US,1635106852,
50,50,Port Blair,11.6667,92.7500,79.92,80,68,4.59,IN,1635106985,
...,...,...,...,...,...,...,...,...,...,...,...
545,545,Esim,4.8699,-2.2405,78.06,86,75,7.20,GH,1635107188,
558,558,La Palma,33.8464,-118.0467,76.12,65,40,3.00,US,1635107192,
563,563,Aswān,24.0934,32.9070,76.30,27,0,9.22,EG,1635107056,
564,564,Kingsport,36.5484,-82.5618,79.03,41,40,3.00,US,1635107195,


In [10]:
#Establish the base_url and params.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [11]:

print("Beginning Data Retrieval")
print("-----------------------------")

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_ideal = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_ideal}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # Add the hotel name to the dataframe. Do a try/except to add a message if there is not hotel for that entry.
    try:
        print(f"Closest hotel in {city_ideal} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("There is no hotel in this city.")
        
    print("------------")
    
    time.sleep(1)

print("Search end.")

Beginning Data Retrieval
-----------------------------
Retrieving Results for Index 8: Bandarbeyla.
Closest hotel in Bandarbeyla is JABIR HOTEL.
------------
Retrieving Results for Index 10: Oriximiná.
Closest hotel in Oriximiná is Hotel Mapuera.
------------
Retrieving Results for Index 27: Kapaa.
Closest hotel in Kapaa is The ISO.
------------
Retrieving Results for Index 31: Hilo.
Closest hotel in Hilo is Grand Naniloa Hotel Hilo - a DoubleTree by Hilton.
------------
Retrieving Results for Index 50: Port Blair.
Closest hotel in Port Blair is SeaShell, Port Blair.
------------
Retrieving Results for Index 52: Galesong.
There is no hotel in this city.
------------
Retrieving Results for Index 53: Djibo.
There is no hotel in this city.
------------
Retrieving Results for Index 57: Saint George.
Closest hotel in Saint George is Red Lion Hotel & Conference Center St. George.
------------
Retrieving Results for Index 58: Russell.
There is no hotel in this city.
------------
Retrieving Re

There is no hotel in this city.
------------
Retrieving Results for Index 515: Santa Cruz de la Sierra.
Closest hotel in Santa Cruz de la Sierra is Marriott Santa Cruz de la Sierra Hotel.
------------
Retrieving Results for Index 525: Raja.
There is no hotel in this city.
------------
Retrieving Results for Index 533: Obregon.
Closest hotel in Obregon is Selina Granada.
------------
Retrieving Results for Index 544: Pānsemāl.
Closest hotel in Pānsemāl is Pragati Restaurant And Hotel.
------------
Retrieving Results for Index 545: Esim.
Closest hotel in Esim is Lou Moon Resort - Eco Luxury.
------------
Retrieving Results for Index 558: La Palma.
Closest hotel in La Palma is Radisson Suites Hotel Anaheim - Buena Park.
------------
Retrieving Results for Index 563: Aswān.
Closest hotel in Aswān is Sofitel Legend Old Cataract Aswan.
------------
Retrieving Results for Index 564: Kingsport.
Closest hotel in Kingsport is Holiday Inn Express & Suites Kingsport-Meadowview I-26, an IHG Hotel.


In [12]:
# Get the new dataframe
hotel_df
new_hotel_df = hotel_df.drop(columns=['Unnamed: 0', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Date'])
new_hotel_df

,City,Lat,Lng,Country,Hotel Name
8,Bandarbeyla,9.4942,50.8122,SO,JABIR HOTEL
10,Oriximiná,-1.7656,-55.8661,BR,Hotel Mapuera
27,Kapaa,22.0752,-159.3190,US,The ISO
31,Hilo,19.7297,-155.0900,US,Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
50,Port Blair,11.6667,92.7500,IN,"SeaShell, Port Blair"
...,...,...,...,...,...
545,Esim,4.8699,-2.2405,GH,Lou Moon Resort - Eco Luxury
558,La Palma,33.8464,-118.0467,US,Radisson Suites Hotel Anaheim - Buena Park
563,Aswān,24.0934,32.9070,EG,Sofitel Legend Old Cataract Aswan
564,Kingsport,36.5484,-82.5618,US,Holiday Inn Express & Suites Kingsport-Meadowv...


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_hotel_df.iterrows()]
locations = new_hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))